# T-SiamTPN Kaggle Notebook
Setup, train 1 epoch, and benchmark on public_test.


In [ ]:
# Install minimal dependencies (Kaggle often preinstalls torch/torchvision)
!pip -q install timm pyyaml opencv-python pandas psutil


In [ ]:
# Verify project files; if running on Kaggle, copy from /kaggle/input/uav if present
import os, shutil, glob, zipfile
ROOT = "/kaggle/working" if os.path.isdir("/kaggle/working") else os.getcwd()
os.chdir(ROOT)
if os.path.isdir("/kaggle/input/uav"):
    # Copy repo contents into working
    for p in glob.glob("/kaggle/input/uav/*"):
        name = os.path.basename(p)
        dst = os.path.join(ROOT, name)
        if os.path.isdir(p):
            if os.path.exists(dst):
                shutil.rmtree(dst)
            shutil.copytree(p, dst)
        else:
            shutil.copy2(p, dst)
if os.path.isfile("observing.zip") and not os.path.isdir("observing/train"):
    print("Unzipping observing.zip ...")
    with zipfile.ZipFile("observing.zip", "r") as z:
        z.extractall(".")
print("Project root:", os.getcwd())
print("Files:", sorted(os.listdir("."))[:20])


In [ ]:
# Patch config for Kaggle environment: enable CUDA if available, adjust workers/batch
import yaml, torch
cfg_path = os.path.join("configs", "default.yaml")
with open(cfg_path, "r") as f:
    cfg = yaml.safe_load(f)
cuda_ok = torch.cuda.is_available()
cfg.setdefault("device", {})
cfg["device"]["use_cuda"] = bool(cuda_ok)
cfg["device"]["mixed_precision"] = bool(cuda_ok)
cfg.setdefault("training", {})
cfg["training"]["num_workers"] = 2
cfg["training"]["batch_size"] = min(int(cfg["training"].get("batch_size", 16)), 8)
with open(cfg_path, "w") as f:
    yaml.safe_dump(cfg, f)
print("CUDA available:", cuda_ok)


In [ ]:
# Train for 1 epoch to produce a checkpoint
import subprocess
subprocess.run(["python", "train.py", "--config", "configs/default.yaml", "--epochs", "1"], check=True)


In [ ]:
# Benchmark on public_test headlessly and write CSV report
import subprocess, os
os.makedirs("reports", exist_ok=True)
subprocess.run(["python", "tools/benchmark.py", "--videos_root", "public_test/samples", "--report", "reports/public_test_benchmark.csv", "--max_frames", "200"], check=True)


In [ ]:
# Display benchmark report
import pandas as pd
df = pd.read_csv("reports/public_test_benchmark.csv")
df.head()
